In [1]:
from preprocess import train, val, train_steps, val_steps, test, test_steps
from keras.applications import Xception
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import callbacks, optimizers
# Uneven class distribution
class_weight = {
    train.class_indices['normal']:1,
    train.class_indices['glaucoma']: 1.86
}

# Using pretrained Xception Net as Convolutional feature extractor
conv_base = Xception(include_top=False,weights=None,input_shape=(299,299,3))
conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizers.RMSprop(1e-5),'binary_crossentropy',metrics=['accuracy'])
print(model.summary())


for layer in conv_base.layers:
    layer.trainable = False

model.compile(optimizers.RMSprop(1e-5),'binary_crossentropy',metrics=['accuracy'])
final_weights_path = '/home/ally/glaucoma-1/weights.h5'
callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        
    ]
model.fit_generator(train,train_steps,30,callbacks=callbacks_list,validation_data=val,validation_steps=val_steps, class_weight=class_weight)

# Save to output
model.save('/home/ally/glaucoma-1/output/trained_dense.h5')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 381 images belonging to 2 classes.
Found 97 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               52429056  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 73,290,793
Trainable params: 52,429,313
Non-trainable params: 20,861,480
_________________________________________________________________


Epoch 29/30
29/30 [============================>.] - ETA: 1s - loss: 0.7286 - acc: 0.6905
Epoch 00029: val_acc did not improve
30/30 [==============================] - 42s 1s/step - loss: 0.7232 - acc: 0.6945 - val_loss: 0.6933 - val_acc: 0.3628
Epoch 30/30
29/30 [============================>.] - ETA: 1s - loss: 0.7759 - acc: 0.6832
Epoch 00030: val_acc did not improve
30/30 [==============================] - 48s 2s/step - loss: 0.7812 - acc: 0.6812 - val_loss: 0.6933 - val_acc: 0.3363


In [6]:
model.load_weights('/home/ally/glaucoma-1/output/trained_dense.h5')
print("Test Accuracy:{}".format(model.evaluate_generator(test, test_steps)[1]))

Test Accuracy:0.351851852956


In [1]:
from preprocess import train, val, train_steps, val_steps, test, test_steps
from keras.applications import Xception
from keras import layers, models
from keras import callbacks, optimizers
# Uneven class distribution
class_weight = {
    train.class_indices['normal']:1,
    train.class_indices['glaucoma']: 1.86
}

# Using pretrained Xception Net as Convolutional feature extractor
conv_base = Xception(include_top=False,weights=None,input_shape=(299,299,3))
conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizers.RMSprop(1e-5),'binary_crossentropy',metrics=['accuracy'])
print(model.summary())

for layer in conv_base.layers:
    layer.trainable = False


print("Loading pre-finetune weights...")
model.load_weights('/home/ally/glaucoma-1/output/trained_dense.h5')
print("Test Accuracy:{}".format(model.evaluate_generator(test, test_steps)[1]))

print("Loading pre-finetune Best Validation weights...")
model.load_weights('/home/ally/glaucoma-1/weights.h5')
print("Test Accuracy:{}".format(model.evaluate_generator(test, test_steps)[1]))


conv_base.trainable = True
for layer in conv_base.layers:
    if layer.name != 'block14_sepconv2':
        layer.trainable = False

model.compile(optimizers.RMSprop(1e-5),'binary_crossentropy',metrics=['accuracy'])

model.fit_generator(train,train_steps,30,callbacks = [callbacks.TensorBoard(log_dir='/home/ally/glaucoma-1/Graph', histogram_freq=0,  
          write_graph=True, write_images=True), callbacks.ModelCheckpoint('/home/ally/glaucoma-1/output/best_weights.h5', save_best_only=True, verbose=1)],validation_data=val,validation_steps=val_steps, class_weight=class_weight)

model.save('/home/ally/glaucoma-1/output/fine.h5')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 381 images belonging to 2 classes.
Found 97 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               52429056  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 73,290,793
Trainable params: 52,429,313
Non-trainable params: 20,861,480
_________________________________________________________________


29/30 [============================>.] - ETA: 1s - loss: 0.7251 - acc: 0.7372
Epoch 00028: val_loss did not improve
30/30 [==============================] - 45s 2s/step - loss: 0.7301 - acc: 0.7335 - val_loss: 0.4589 - val_acc: 0.8061
Epoch 29/30
29/30 [============================>.] - ETA: 1s - loss: 0.6400 - acc: 0.7662
Epoch 00029: val_loss did not improve
30/30 [==============================] - 39s 1s/step - loss: 0.6572 - acc: 0.7636 - val_loss: 0.4663 - val_acc: 0.8053
Epoch 30/30
29/30 [============================>.] - ETA: 1s - loss: 0.6877 - acc: 0.7586
Epoch 00030: val_loss did not improve
30/30 [==============================] - 46s 2s/step - loss: 0.6753 - acc: 0.7646 - val_loss: 0.4519 - val_acc: 0.8142


In [7]:
print("Test Accuracy:{}".format(model.evaluate_generator(test, test_steps)[0]))

print("Loading Fine tuned Best Validation weights...")
model.load_weights('/home/ally/glaucoma-1/output/best_weights.h5')
print("Test Accuracy:{}".format(model.evaluate_generator(test, test_steps)[1]))

Test Accuracy:0.693241108347
Loading Fine tuned Best Validation weights...
Test Accuracy:0.648148150356


In [1]:
from preprocess import train, val, train_steps, val_steps, test, test_steps
from keras.applications import Xception
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import callbacks, optimizers
# Uneven class distribution
class_weight = {
    train.class_indices['normal']:1,
    train.class_indices['glaucoma']: 1.86
}

# Using pretrained Xception Net as Convolutional feature extractor
conv_base = Xception(include_top=False,weights=None,input_shape=(299,299,3))
conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizers.RMSprop(1e-5),'binary_crossentropy',metrics=['accuracy'])
#print(model.summary())

#print(model.summary())
print("Loading weights...")
model.load_weights('/home/ally/glaucoma-1/weights.h5')
print("Evaluating on test data")
print(model.evaluate_generator(test, test_steps))

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 381 images belonging to 2 classes.
Found 97 images belonging to 2 classes.
Found 54 images belonging to 2 classes.
Loading weights...


ValueError: Dimension 2 in both shapes must be equal, but are 3 and 1536 for 'Assign' (op: 'Assign') with input shapes: [3,3,3,32], [3,3,1536,1].